In [1]:
import random

In [14]:
query1='''
SELECT i_item_id, 
               Avg(ss_quantity)    agg1, 
               Avg(ss_list_price)  agg2, 
               Avg(ss_coupon_amt)  agg3, 
               Avg(ss_sales_price) agg4 
FROM   tpcds.dbo.store_sales, 
       tpcds.dbo.customer_demographics, 
       tpcds.dbo.date_dim, 
       tpcds.dbo.item, 
       tpcds.dbo.promotion 
WHERE  ss_sold_date_sk = d_date_sk 
       AND ss_item_sk = i_item_sk 
       AND ss_cdemo_sk = cd_demo_sk 
       AND ss_promo_sk = p_promo_sk 
       AND cd_gender = '{{GENDER}}'
       AND cd_marital_status = '{{MARITAL_STAT}}'
       AND cd_education_status = '{{EDU_STAT}}'
       AND ( p_channel_email = 'N' 
              OR p_channel_event = 'N' ) 
       AND d_year = 1998 
GROUP  BY i_item_id 
ORDER  BY i_item_id
'''
gender=['M','F']
marital=['U','S','D','M','W']
education=['Unknown','Primary','Secondary','4 yr Degree','Advanced Degree','College','2 yr Degree']

for i in range(50):
    g=random.choice(gender)
    m=random.choice(marital)
    e=random.choice(education)
    query=query1.replace('{{GENDER}}',g).replace('{{MARITAL_STAT}}',m).replace('{{EDU_STAT}}',e)
    f=open(r"G:\dbquery-optimizer\tpcds\query_variants\query_1\q_one_"+str(i+1)+".sql",'w')
    f.write(query)
    f.close()



In [15]:
query2='''
SELECT dt.d_year, 
               item.i_brand_id          brand_id, 
               item.i_brand             brand, 
               Sum(ss_ext_discount_amt) sum_agg 
FROM   tpcds.dbo.date_dim dt, 
       tpcds.dbo.store_sales, 
       tpcds.dbo.item 
WHERE  dt.d_date_sk = store_sales.ss_sold_date_sk 
       AND store_sales.ss_item_sk = item.i_item_sk 
       AND item.i_manufact_id = {{MF_ID}}
       AND dt.d_moy = {{MONTH}}
GROUP  BY dt.d_year, 
          item.i_brand, 
          item.i_brand_id 
ORDER  BY dt.d_year, 
          sum_agg DESC, 
          brand_id
'''

for i in range(50):
    id=random.randint(1,1000)
    month=random.randint(1,12)
    query=query2.replace('{{MF_ID}}',str(id)).replace('{{MONTH}}',str(month))
    f=open(r"G:\dbquery-optimizer\tpcds\query_variants\query_2\q_two_"+str(i+1)+".sql",'w')
    f.write(query)
    f.close()



In [ ]:
query3='''
SELECT i_item_id, 
               s_state, 
               Grouping(s_state)   g_state, 
               Avg(ss_quantity)    agg1, 
               Avg(ss_list_price)  agg2, 
               Avg(ss_coupon_amt)  agg3, 
               Avg(ss_sales_price) agg4 
FROM   store_sales, 
       customer_demographics, 
       date_dim, 
       store, 
       item 
WHERE  ss_sold_date_sk = d_date_sk 
       AND ss_item_sk = i_item_sk 
       AND ss_store_sk = s_store_sk 
       AND ss_cdemo_sk = cd_demo_sk 
       AND cd_gender = 'M' 
       AND cd_marital_status = 'D' 
       AND cd_education_status = 'College' 
       AND d_year = 2000 
       AND s_state IN ( 'TN', 'TN', 'TN', 'TN', 
                        'TN', 'TN' ) 
GROUP  BY rollup ( i_item_id, s_state ) 
ORDER  BY i_item_id, 
          s_state
'''